This script categorize the VOC measured from redlands to SAPRC11 species

In [1]:
import os, importlib, sys
import pandas as pd
import numpy as np

In [21]:
df_VOC_CARB_Chamber = pd.read_csv('../../data/raw/df_VOC_CARB_Chamber_DNPHcorrected.csv')
df_VOC_CARB_Chamber['Sample date'] = pd.to_datetime(df_VOC_CARB_Chamber['Sample date'])
df_VOC_redlands = df_VOC_CARB_Chamber[df_VOC_CARB_Chamber['Type']=='VOC']
df_saprc_name_redlands = pd.read_csv('../../data/raw/VOC_saprc_name_redlands.csv')

In [22]:
# correct HCHO and CCHO
df_VOC_redlands.loc[df_VOC_redlands['Parameter']=='Formaldehyde', ('Value')]=\
    df_VOC_redlands.loc[df_VOC_redlands['Parameter']=='Formaldehyde', ('Value')]*4.165*0.9
df_VOC_redlands.loc[df_VOC_redlands['Parameter']=='Acetaldehyde', ('Value')]=\
    df_VOC_redlands.loc[df_VOC_redlands['Parameter']=='Acetaldehyde', ('Value')]*6.00*0.66

# remove Isopropanol (unrealistically large )
df_VOC_redlands.loc[df_VOC_redlands['Parameter']=='Isopropanol', ('Value')]=np.nan

c:\Users\Shenglun\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
c:\Users\Shenglun\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [10]:
# categorize VOCs into SAPRC species and save df_saprc_redlands
df_VOC_saprc = df_VOC_redlands.\
    merge(df_saprc_name_redlands[['Parameter','SAPRC']], on='Parameter', how='left')
df_VOC_saprc = df_VOC_saprc.groupby(['Sample date','SAPRC']).\
    agg({'Value':'sum'}).reset_index()
df_VOC_saprc['Value'] = df_VOC_saprc['Value']

# fill NaN concentration as LOD/3 (LOD = min(conc)/3) and constran large MeOH
for i in df_VOC_saprc['SAPRC'].unique():
    conc_min = np.min(df_VOC_saprc[(df_VOC_saprc['SAPRC']==i)&
                                   (df_VOC_saprc['Value']!=0)]['Value'])
    conc_max = np.max(df_VOC_saprc[(df_VOC_saprc['SAPRC']==i)&(df_VOC_saprc['Value']!=0)&
                                   (df_VOC_saprc['Value']>500)]['Value'])
    
    df_VOC_saprc.loc[(df_VOC_saprc['SAPRC']==i)&
                     (~(df_VOC_saprc['Value']>0)), 'Value'] = conc_min/3
    df_VOC_saprc.loc[(df_VOC_saprc['SAPRC']=='MEOH')&
                     (df_VOC_saprc['Value']>500), 'Value'] = conc_max

df_VOC_saprc.columns = ['Date','S11','Value']
df_saprc_redlands = df_VOC_saprc
df_saprc_redlands.to_csv('../../data/intermediate/saprc11_initial_conc_redlands.csv',
                          index=False)

## PTR-MS measured VOC

In [9]:
df_PTR_redlands = pd.read_csv('../../data/raw/CARB_PTR_redlands.txt')
df_PTR_redlands['dates'] = pd.to_datetime(df_PTR_redlands['dates'])

df_temp_redlands = pd.read_csv('../../data/final/temp_morning_daily_avg_redlands.csv')
df_temp_redlands['Date'] = pd.to_datetime(df_temp_redlands['Date'])

In [10]:
# fill NAN and 0 by min/3
for i in df_PTR_redlands.columns[1:]:
    min_value = min(df_PTR_redlands.loc[df_PTR_redlands[i]>0,i])   
    df_PTR_redlands.loc[~(df_PTR_redlands[i]>0),i] = min_value/3

In [11]:
df_PTR_redlands.to_csv('../../data/final/CARB_PTR_redlands.csv', index=False)

# Get list of all VOCs

In [4]:
df_saprc_redlands = pd.read_csv('../../data/intermediate/saprc11_initial_conc_redlands.csv')
df_PTR_redlands = pd.read_csv('../../data/final/CARB_PTR_redlands.csv')

In [19]:
# reshape PTR data
df_melt = df_PTR_redlands.melt(id_vars=['dates'], var_name='Parameter', value_name='Value')
df_melt.groupby('Parameter').agg({'Value':['min','max','mean','std','median']})

Value                                        
                       min       max      mean       std    median
Parameter                                                         
Acetone           1.411143  8.309027  4.042603  1.674688  3.804326
Acetonitrile      0.116745  0.285546  0.176158  0.035255  0.172290
Acrylonitrile     0.000014  0.010235  0.001235  0.002576  0.000014
Benzene           0.000053  3.021173  0.177358  0.561231  0.022152
D4                0.000171  0.008493  0.002457  0.001755  0.002451
D5                0.000424  0.021805  0.008291  0.005705  0.007814
Ethanol           0.637515  7.841174  3.980247  2.289900  3.838503
Isoprene          0.114864  1.131562  0.588730  0.259808  0.624298
MEK               0.000335  8.798581  0.812687  1.980416  0.005774
MVK               0.030149  1.539817  0.335372  0.259159  0.297522
Trimethylbenzene  0.005196  0.252511  0.068178  0.056839  0.060309
Xylene            0.024997  0.449438  0.166238  0.098639  0.157707
a-Pinene          0.048977  0.254069  0.141292  0.054529  0.133166
b-Caryophyllene   0.006388  0.073305  0.022723  0.014681  0.021245

In [24]:
# groupby Parameter and agg to get min, max, mean, std, median of Value
df_VOC_redlands.groupby('Parameter').agg(['min','max','mean','std','median']).reset_index().to_csv('../../data/final/VOC_stat_redlands_correct-HCHO-CCHO.csv', index=False)